# Copy instead of modifying to preserve attack parameters

In [14]:
AGENT_NAME = 'Models\ATLA\PPO agent 100 alts over 1000+200 2-3-21.zip'
DATASET_NAME = 'citylearn_challenge_2022_phase_2' #only action is electrical storage
#RUNS = 5
TRIAL = 1
SAVE_DIR = r'PPO agent 100 alts over 1000+200 2-3-21 results\random noise eps vs ASR results' + '/'

In [2]:
from stable_baselines3 import PPO

from citylearn.data import DataSet

import pandas as pd
import numpy as np


import KBMproject.utilities as utils

from joblib import Parallel, delayed

%matplotlib inline

In [3]:
schema = DataSet.get_schema(DATASET_NAME)

In [4]:
testilons = np.arange(0.02, 0.21, 0.01)
RUNS = len(testilons)

Define RL agent

In [5]:
agents = []
for _ in range (RUNS):
    agents.append(PPO.load(AGENT_NAME))

Create environments

In [6]:
envs = []
for _ in range (RUNS):
    envs.append(utils.make_discrete_env(schema=schema,  
                            action_bins=agents[0].action_space[0].n,
                            seed=42))

In [7]:
cols = utils.make_discrete_env(schema=schema,  
                            action_bins=agents[0].action_space[0].n,
                            seed=42).observation_names

In [8]:
observation_masks = np.ones(agents[0].observation_space.shape)
observation_masks[0:6] = 0 #mask time features
print('masked features:')
cols[0][0:6]

masked features:


['month_cos',
 'month_sin',
 'day_type_cos',
 'day_type_sin',
 'hour_cos',
 'hour_sin']

In [9]:
%%time
%%capture
results = Parallel(n_jobs=RUNS, verbose=10, prefer='threads')(delayed(
    utils.eval_rand_attack)(agent, env, eps) for agent, env, eps in zip(agents, envs, testilons)) 


CPU times: total: 2min 32s
Wall time: 11min 22s


Results is a list of tupples for each run, of the format(KPIs, observations, perturbed observations, epsilons)

In [10]:
kpis = [results[i][0] for i in range(len(results))]
df_kpis = pd.concat(kpis, axis='columns',keys=testilons)

In [11]:
#df_kpis[['mean', 'std', 'variance']] = df_kpis.agg(['mean','std', 'var'], axis='columns')

In [12]:
df_kpis

,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.10,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.20
cost_function,,,,,,,,,,,,,,,,,,,
annual_peak_average,1.042875,1.042875,1.042875,1.042875,1.042875,1.042875,1.042875,1.042875,1.042875,1.042875,1.042875,0.938806,1.042875,0.932270,1.042875,1.042875,1.009986,1.009986,1.300127
carbon_emissions_total,0.893533,0.892528,0.894098,0.892676,0.893362,0.892589,0.893780,0.892408,0.893138,0.892371,0.894980,0.893062,0.892748,0.893269,0.892154,0.893455,0.894547,0.893679,0.894411
cost_total,0.803288,0.801791,0.803739,0.800628,0.803135,0.801149,0.802031,0.800832,0.801877,0.800729,0.802775,0.801351,0.801052,0.799831,0.798256,0.800065,0.800635,0.798445,0.800797
daily_one_minus_load_factor_average,1.028845,1.029023,1.031406,1.033628,1.026790,1.031051,1.029084,1.036343,1.027288,1.021681,1.020444,1.029489,1.026867,1.116876,1.022912,1.024533,1.028018,1.115700,1.023970
daily_peak_average,0.917629,0.919160,0.925961,0.918449,0.923137,0.921833,0.924508,0.918737,0.918348,0.921210,0.928157,0.920760,0.928104,0.920908,0.920697,0.919241,0.913960,0.917974,0.926489
electricity_consumption_total,0.907258,0.906459,0.907733,0.906908,0.907476,0.906905,0.908127,0.907011,0.907283,0.907172,0.909493,0.908078,0.907546,0.908559,0.906957,0.908527,0.909869,0.909016,0.909949
monthly_one_minus_load_factor_average,0.987542,0.987721,0.987294,0.981711,0.980945,0.986775,0.989473,0.982958,0.978747,0.982595,0.985580,0.983017,0.984916,0.984601,0.986055,0.984599,0.976182,0.983781,0.990219
ramping_average,1.113565,1.115423,1.116467,1.115488,1.116296,1.114102,1.119726,1.113795,1.114810,1.108789,1.122712,1.119892,1.121215,1.120157,1.116672,1.116370,1.119659,1.107117,1.124055
zero_net_energy,1.102841,1.102860,1.102881,1.103127,1.103135,1.103533,1.103556,1.103956,1.103459,1.103846,1.104469,1.104825,1.104607,1.104742,1.105609,1.104847,1.105017,1.105664,1.105373


In [15]:
df_kpis.to_csv(SAVE_DIR + f'{RUNS} run KPIs {TRIAL}.csv')

FGSM using the same parameters in a whitebox attack has an ASR of 0.7.

Increasing the number of restarts decreases the ASR, likely because of differences between the surrogate and vicitm models

In [16]:
ASRs = [results[i][3] for i in range(len(results))]
#print(f'For 5 runs of the random attack \nthe mean ASR is: {np.mean(ASRs):.3f}\nthe STD is: {np.std(ASRs):.3f}')

In [17]:
ASRs

[0.06244292237442922,
 0.0930365296803653,
 0.12351598173515982,
 0.1519406392694064,
 0.18538812785388128,
 0.20319634703196346,
 0.22442922374429225,
 0.24942922374429224,
 0.26860730593607307,
 0.2787671232876712,
 0.3057077625570776,
 0.3232876712328767,
 0.3329908675799087,
 0.35182648401826483,
 0.37123287671232874,
 0.37431506849315066,
 0.39794520547945206,
 0.4046803652968037,
 0.42077625570776256]

In [18]:
pd.DataFrame({'ASRs':ASRs}, index=testilons,).to_csv(SAVE_DIR + f'ASR for varied epsilons {TRIAL}.csv')

In [19]:
for run in range(RUNS):
        pd.DataFrame(results[run][1],columns=cols,).to_csv(SAVE_DIR+f'run {run} obs {TRIAL}.csv',)
        pd.DataFrame(results[run][2],columns=cols,).to_csv(SAVE_DIR+f'run {run} adv obs {TRIAL}.csv')

ValueError: Shape of passed values is (8759, 1), indices imply (8759, 31)